In [1]:
cd /home/kuhlmanlab/Documents/GitHub/

/home/kuhlmanlab/Documents/GitHub


In [2]:
import luriadelbruck as ldb
import numpy as np

This experiment used 3 different inducer combinations to induce different amounts of mutH. All cultures had 2 mM iptg. The aTc concentrations were .01, .1, and 1 ng/ml respectively. There were 10 replicates for each condition

In [3]:
aTcp01_rif = np.array([39,20,58,146,54,80,120,67,13,209])
aTcp1_rif = np.array([33,29,63,23,32,31,21,66,6,36])
aTc1_rif = np.array([1,5,17,4,0,6,4,4,2,41])
aTcp01_LB = np.array([37,51,38,911,49,70,90,28,50,34])*2.5*10**6
aTcp1_LB = np.array([120,50,66,46,40,106,53,94,82,55])*2.5*10**6
aTc1_LB = np.array([65,61,53,89,84,57,53,82,80,72])*2.5*10**6

In [4]:
sample1_inference =ldb.SarkarMaSandri(aTcp01_rif, aTcp01_LB)
sample2_inference =ldb.SarkarMaSandri(aTcp1_rif, aTcp1_LB)
sample3_inference =ldb.SarkarMaSandri(aTc1_rif, aTc1_LB)
results1 = sample1_inference.fit()
results2 = sample2_inference.fit()
results3 = sample3_inference.fit()
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .01 ng/ml aTc is', results1.params[0])
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .1 ng/ml aTc is', results2.params[0])
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and 1 ng/ml aTc is', results3.params[0])

Optimization terminated successfully.
         Current function value: 75.202774
         Iterations: 11
         Function evaluations: 22


/home/kuhlmanlab/Documents/GitHub/luriadelbruck.py:100: RuntimeWarning: divide by zero encountered in log
  self.exog)))
/home/kuhlmanlab/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


Optimization terminated successfully.
         Current function value: 46.367762
         Iterations: 15
         Function evaluations: 30
Optimization terminated successfully.
         Current function value: 31.438278
         Iterations: 15
         Function evaluations: 30
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .01 ng/ml aTc is 1.72227561834e-08
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .1 ng/ml aTc is 4.86393929929e-08
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and 1 ng/ml aTc is 1.24031732456e-08


These result for the .01 ng/ml aTc condition is very strange. It is lower than that for .1 ng/ml aTc. The culture with 911 colonies when plated on LB is an obvious outlier though. It's an order of magnitude away from every other count on LB. It's not obvious exactly what could have gone wrong, but it may have had a strong effect on the results. If we remove the data for this culture, and then re-estimate the mutation rate...

In [5]:
aTcp01_rif_cl = np.array([39,20,58,54,80,120,67,13,209])
aTcp01_LB_cl = np.array([37,51,38,49,70,90,28,50,34])*2.5*10**6
sample1_inference_cl = ldb.SarkarMaSandri(aTcp01_rif_cl, aTcp01_LB_cl)
results1_cl = sample1_inference_cl.fit()
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .01 ng/ml aTc is', results1_cl.params[0])
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .1 ng/ml aTc is', results2.params[0])
print('the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and 1 ng/ml aTc is', results3.params[0])

Optimization terminated successfully.
         Current function value: 49.242507
         Iterations: 12
         Function evaluations: 24
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .01 ng/ml aTc is 9.74456295253e-08
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and .1 ng/ml aTc is 4.86393929929e-08
the estimated mutation rate of ME121 Ptet-CH lacI RBS pTKIP-tetR with 2mM iptg and 1 ng/ml aTc is 1.24031732456e-08


/home/kuhlmanlab/Documents/GitHub/luriadelbruck.py:100: RuntimeWarning: divide by zero encountered in log
  self.exog)))
/home/kuhlmanlab/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


These results make way more sense. I'll have to think more carefully about how to exclude outliers. The number of colonies on LB should be approximately a lognormal (growth process) convoluted with a poisson distribution (small sample from a very large culture), so it probably makes sense to exclude anything far out.